In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from surprise import Dataset, Reader, accuracy
from sklearn.metrics.pairwise import cosine_similarity
from surprise.model_selection import train_test_split
from surprise import SVD
import numpy as np
import pandas as pd

# Preprocessing

In [2]:
# Load the data
data_raw = pd.read_csv('Fix_Dataset/asuransi_clean.csv')

In [3]:
data_raw.head()

,created_at,full_text,username,Produk,rate
0,2023-09-25,"['1', '5', 'juta', 'sist', 'kantor', 'sih', 'j...",Ditanyadia,AIA,2
1,2023-09-22,"['mohon', 'klarifikasi', 'dari', 'asuransi', '...",MuhTaufikRahmat,AIA,2
2,2023-09-21,"['indonesia', 'unit', 'link', '5', 'tips', 'se...",fiona99351131,AIA,2
3,2023-09-21,"['halo', 'bang', 'maaf', 'nimbrung', 'pesan', ...",fiayeee,AIA,2
4,2023-09-18,"['impactnya', 'rumah', 'sakit', 'rekan', 'reka...",filcadet,AIA,1


In [4]:
# membuat user_id dari username dengan encoding labelencoder supaya unik dan hanya dimiliki oleh satu user saja
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

data_raw['user_id'] = le.fit_transform(data_raw['username'])

data_raw.head()

,created_at,full_text,username,Produk,rate,user_id
0,2023-09-25,"['1', '5', 'juta', 'sist', 'kantor', 'sih', 'j...",Ditanyadia,AIA,2,138
1,2023-09-22,"['mohon', 'klarifikasi', 'dari', 'asuransi', '...",MuhTaufikRahmat,AIA,2,295
2,2023-09-21,"['indonesia', 'unit', 'link', '5', 'tips', 'se...",fiona99351131,AIA,2,816
3,2023-09-21,"['halo', 'bang', 'maaf', 'nimbrung', 'pesan', ...",fiayeee,AIA,2,813
4,2023-09-18,"['impactnya', 'rumah', 'sakit', 'rekan', 'reka...",filcadet,AIA,1,815


In [5]:
data_raw['user_id'].value_counts().head()

user_id
268     148
1023    125
186      99
66       90
64       89
Name: count, dtype: int64

In [6]:
# mengubah rate menjadi 1-5
data_raw['rate'] = data_raw['rate'].map({0: 1, 1: 2, 2: 3, 3: 4, 4: 5})

In [7]:
data_raw['rate'].value_counts()

rate
2    1897
3    1159
4     126
1      36
5       3
Name: count, dtype: int64

In [8]:
df = data_raw.copy()

In [9]:
# encoding data pada kolom Produk
label = { 'AIA' : 0, 'Allianz' : 1, 'Prudential' : 2, 'BNI Life' : 3, 'Manulife' : 4, 'Cigna' : 5 }

df['label'] = df['Produk'].replace(label)

In [10]:
df.head()

,created_at,full_text,username,Produk,rate,user_id,label
0,2023-09-25,"['1', '5', 'juta', 'sist', 'kantor', 'sih', 'j...",Ditanyadia,AIA,3,138,0
1,2023-09-22,"['mohon', 'klarifikasi', 'dari', 'asuransi', '...",MuhTaufikRahmat,AIA,3,295,0
2,2023-09-21,"['indonesia', 'unit', 'link', '5', 'tips', 'se...",fiona99351131,AIA,3,816,0
3,2023-09-21,"['halo', 'bang', 'maaf', 'nimbrung', 'pesan', ...",fiayeee,AIA,3,813,0
4,2023-09-18,"['impactnya', 'rumah', 'sakit', 'rekan', 'reka...",filcadet,AIA,2,815,0


# Modelling

In [11]:
# langkah pertama sebelum membuat model Collaborative Filtering adalah membuat data menjadi format yang sesuai dengan library surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'label', 'rate']], reader)

# split data menjadi data train dan data test
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# membuat model Collaborative Filtering dengan menggunakan SVD
model = SVD(n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.02)

In [12]:
# training model
model.fit(trainset)

In [13]:
# membuat prediksi
predictions = model.test(testset)

In [14]:
# menghitung error model dengan menggunakan MAE
mae = accuracy.mae(predictions)
print('MAE: ', mae)

MAE:  0.4904
MAE:  0.49036900549356954


In [16]:
# membuat fungsi untuk merekomendasikan produk asuransi
def recommend_product(user_id, model, n=5):
    # membuat list produk yang sudah direview oleh user
    produk_reviewed = df[df['user_id'] == user_id]['label'].unique()
    
    # membuat list produk yang belum direview oleh user
    produk_not_reviewed = [produk for produk in df['label'].unique() if produk not in produk_reviewed]
    
    # membuat prediksi rating untuk produk yang belum direview oleh user
    predictions = [model.predict(user_id, produk) for produk in produk_not_reviewed]
    
    # mengurutkan produk berdasarkan prediksi rating
    predictions.sort(key=lambda x: x.est, reverse=True)
    
    # mengambil n produk teratas
    top_n = predictions[:n]
    
    # mendapatkan id produk
    top_n_produk = [pred.iid for pred in top_n]
    
    # mendapatkan nama produk
    top_n_produk_name = [df[df['label'] == produk]['Produk'].values[0] for produk in top_n_produk]
    
    return top_n_produk_name

In [17]:
# membuat rekomendasi produk untuk user dengan id 
user_id = 15
recommend_product(user_id, model)

['Cigna', 'Prudential', 'AIA', 'Manulife', 'Allianz']

In [18]:
# menggunakan text review untuk merekomendasikan produk asuransi
# membuat model TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data_raw['full_text'])

In [63]:
# membuat fungsi untuk merekomendasikan produk asuransi berdasarkan text review membandingkan word2vec dan tfidf
# untuk text review yang diberikan oleh user menggunakan tfidf
def recommend_product_tfidf(text_review, model, n=5):
    # membuat vektor tfidf dari text review yang diberikan oleh user
    tfidf_vector = tfidf.transform([text_review])
    
    # menghitung cosine similarity antara vektor tfidf user dengan vektor tfidf produk
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_vector)
    
    # mengurutkan produk berdasarkan cosine similarity
    produk_index = cosine_sim.flatten().argsort()[::-1]
    
    # mengambil n produk teratas
    top_n = produk_index[:n]
    
    # mendapatkan nama produk
    top_n_produk_name = data_raw.iloc[top_n]['Produk'].unique()
    
    return top_n_produk_name

# membuat rekomendasi produk untuk user dengan text review
text_review = 'rekomendasi asuransi yang murah dan bagus untuk keluarga'
recommend_product_tfidf(text_review, model)

array(['BNI Life', 'AIA', 'Allianz'], dtype=object)

In [71]:
# deploy model dengan menggunakan pickle
import pickle

# save model
with open('Model/model_cf.pkl', 'wb') as file:
    pickle.dump(model, file)

# save tfidf
with open('Model/tfidf.pkl', 'wb') as file:
    pickle.dump(tfidf, file)

In [74]:
# test load model
with open('Model/model_cf.pkl', 'rb') as file:
    model_pkl_cf = pickle.load(file)

# test load tfidf
with open('Model/tfidf.pkl', 'rb') as file:
    tfidf_pkl = pickle.load(file)

In [75]:
# membuat rekomendasi produk untuk user dengan id
user_id = 15
recommend_product(user_id, model_pkl_cf)

['Cigna', 'Prudential', 'AIA', 'Manulife', 'Allianz']

In [76]:
# membuat rekomendasi produk untuk user dengan text review
text_review = 'rekomendasi asuransi yang murah dan bagus untuk keluarga'
recommend_product_tfidf(text_review, model_pkl_cf)

array(['BNI Life', 'AIA', 'Allianz'], dtype=object)

In [ ]:
# membuat sistem rekomendasi produk asuransi menggunakan streamlit
import streamlit as st
import pickle

# load model
with open('Model/model_cf.pkl', 'rb') as file:
    model_pkl_cf = pickle.load(file)

# load tfidf
with open('Model/tfidf.pkl', 'rb') as file:
    tfidf_pkl = pickle.load(file)

# membuat judul
st.title('Rekomendasi Produk Asuransi')

# membuat input text review
text_review = st.text_input('Masukkan Text Review')

# membuat tombol untuk merekomendasikan produk
if st.button('Rekomendasi Produk'):
    # merekomendasikan produk asuransi berdasarkan text review
    recommended_product = recommend_product_tfidf(text_review, model_pkl_cf)

    # menampilkan rekomendasi produk
    st.write('Rekomendasi Produk:')
    for i, produk in enumerate(recommended_product):
        st.write(f'{i+1}. {produk}')

In [ ]:
# membuat dashboard untuk bagian data eksplorasi dan visualisasi
import matplotlib.pyplot as plt
import seaborn as sns

# membuat judul
st.title('Data Eksplorasi dan Visualisasi')

# menampilkan data eksplorasi
st.write('Data Eksplorasi:')
st.write('Jumlah Data:', len(data_raw))
st.write('Jumlah User:', data_raw['user_id'].nunique())
st.write('Jumlah Produk:', data_raw['Produk'].nunique())

# membuat plot jumlah review per produk
plt.figure(figsize=(10, 6))
sns.countplot(data_raw['Produk'], order=data_raw['Produk'].value_counts().index)
plt.xticks(rotation=45)
plt.title('Jumlah Review per Produk')
plt.tight_layout()
st.pyplot()

# membuat plot jumlah review per user
plt.figure(figsize=(10, 6))
sns.countplot(data_raw['user_id'], order=data_raw['user_id'].value_counts().index)
plt.title('Jumlah Review per User')
plt.tight_layout()
st.pyplot()

# membuat plot distribusi rating
plt.figure(figsize=(10, 6))
sns.countplot(data_raw['rate'])
plt.title('Distribusi Rating')
plt.tight_layout()
st.pyplot()

# membuat plot distribusi rating per produk
plt.figure(figsize=(10, 6))
sns.countplot(data_raw['Produk'], hue=data_raw['rate'], order=data_raw['Produk'].value_counts().index)
plt.xticks(rotation=45)
plt.title('Distribusi Rating per Produk')
plt.tight_layout()
st.pyplot()

# membuat plot distribusi rating per user
plt.figure(figsize=(10, 6))
sns.countplot(data_raw['user_id'], hue=data_raw['rate'], order=data_raw['user_id'].value_counts().index)
plt.title('Distribusi Rating per User')
plt.tight_layout()
st.pyplot()
